# Stable Diffusion WebUI on SageMaker

This notebook helps you manage and interact with Stable Diffusion WebUI on SageMaker.

## 1. Environment Setup

In [ ]:
import subprocess
import os
import time
import requests
import json
from IPython.display import display, HTML
import threading

## 2. Check GPU and System Status

In [ ]:
# Check GPU status
!nvidia-smi

In [ ]:
# Check disk space
!df -h /home/ec2-user/SageMaker

In [ ]:
# Check Python packages
!pip list | grep -E "torch|xformers|diffusers"

## 3. Install/Update Stable Diffusion WebUI

In [ ]:
# Install or update WebUI
webui_path = "/home/ec2-user/SageMaker/stable-diffusion-webui"

if not os.path.exists(webui_path):
    print("Installing Stable Diffusion WebUI...")
    !cd /home/ec2-user/SageMaker && git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
else:
    print("Updating Stable Diffusion WebUI...")
    !cd {webui_path} && git pull

In [ ]:
# Install requirements
!cd {webui_path} && pip install -r requirements.txt

## 4. Download Models

In [ ]:
# Create models directory
models_dir = f"{webui_path}/models/Stable-diffusion"
controlnet_dir = f"{webui_path}/models/ControlNet"

os.makedirs(models_dir, exist_ok=True)
os.makedirs(controlnet_dir, exist_ok=True)

print(f"Models directory: {models_dir}")
print(f"ControlNet directory: {controlnet_dir}")

In [ ]:
# Download EpicRealism model (if not exists)
epic_realism_path = f"{models_dir}/sd1_5-epiCRealism.safetensors"

if not os.path.exists(epic_realism_path):
    print("Downloading EpicRealism model...")
    !cd {models_dir} && wget -O sd1_5-epiCRealism.safetensors "https://huggingface.co/emilianJR/epiCRealism/resolve/main/epicrealism_naturalSinRC1VAE.safetensors"
else:
    print("EpicRealism model already exists")

In [ ]:
# Download ControlNet models
controlnet_models = {
    "control_v11p_sd15_inpaint.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth",
    "control_v11p_sd15_canny.pth": "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth"
}

for model_name, url in controlnet_models.items():
    model_path = f"{controlnet_dir}/{model_name}"
    if not os.path.exists(model_path):
        print(f"Downloading {model_name}...")
        !cd {controlnet_dir} && wget -O {model_name} "{url}"
    else:
        print(f"{model_name} already exists")

## 5. Start WebUI Server

In [ ]:
class WebUIManager:
    def __init__(self, webui_path):
        self.webui_path = webui_path
        self.process = None
        self.port = 8888
        
    def start(self):
        if self.is_running():
            print("WebUI is already running!")
            return
            
        print("Starting Stable Diffusion WebUI...")
        
        # Set environment variables
        env = os.environ.copy()
        env['COMMANDLINE_ARGS'] = f'--listen --port {self.port} --enable-insecure-extension-access --xformers --no-half-vae'
        
        # Start WebUI in background
        os.chdir(self.webui_path)
        self.process = subprocess.Popen(
            ['python', 'launch.py'],
            env=env,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        
        print(f"WebUI starting... Process ID: {self.process.pid}")
        print("Please wait 2-3 minutes for initialization...")
        
        # Wait for server to start
        self.wait_for_server()
        
    def wait_for_server(self, timeout=300):
        """Wait for WebUI server to be ready"""
        start_time = time.time()
        
        while time.time() - start_time < timeout:
            try:
                response = requests.get(f'http://localhost:{self.port}', timeout=5)
                if response.status_code == 200:
                    print("✅ WebUI is ready!")
                    self.display_access_info()
                    return True
            except:
                pass
                
            time.sleep(10)
            print("⏳ Still starting...")
            
        print("❌ Timeout waiting for WebUI to start")
        return False
        
    def is_running(self):
        """Check if WebUI is running"""
        try:
            response = requests.get(f'http://localhost:{self.port}', timeout=5)
            return response.status_code == 200
        except:
            return False
            
    def stop(self):
        """Stop WebUI server"""
        if self.process:
            self.process.terminate()
            self.process.wait()
            print("WebUI stopped")
        else:
            print("WebUI is not running")
            
    def display_access_info(self):
        """Display access information"""
        # Get notebook instance name from metadata
        try:
            import boto3
            session = boto3.Session()
            region = session.region_name
            
            # Try to get instance name from environment or metadata
            instance_name = os.environ.get('SM_CURRENT_INSTANCE_NAME', 'your-notebook-instance')
            
            proxy_url = f"https://{instance_name}.notebook.{region}.sagemaker.aws/proxy/{self.port}/"
            
            display(HTML(f"""
            <div style="border: 2px solid #4CAF50; padding: 20px; border-radius: 10px; background-color: #f9f9f9;">
                <h3 style="color: #4CAF50;">🎉 Stable Diffusion WebUI is Ready!</h3>
                <p><strong>Access URL:</strong></p>
                <p><a href="{proxy_url}" target="_blank" style="font-size: 16px; color: #2196F3;">{proxy_url}</a></p>
                <p><em>Click the link above to open WebUI in a new tab</em></p>
            </div>
            """))
            
        except Exception as e:
            print(f"WebUI is running on port {self.port}")
            print(f"Access via SageMaker proxy URL")

# Create WebUI manager
webui_manager = WebUIManager(webui_path)

In [ ]:
# Start WebUI
webui_manager.start()

## 6. WebUI Status and Control

In [ ]:
# Check WebUI status
if webui_manager.is_running():
    print("✅ WebUI is running")
    webui_manager.display_access_info()
else:
    print("❌ WebUI is not running")

In [ ]:
# Stop WebUI (if needed)
# webui_manager.stop()

## 7. API Usage Examples

In [ ]:
# Example: Generate image via API
def generate_image_api(prompt, negative_prompt="", steps=20, cfg_scale=7):
    """Generate image using WebUI API"""
    
    if not webui_manager.is_running():
        print("WebUI is not running. Please start it first.")
        return None
        
    url = f"http://localhost:{webui_manager.port}/sdapi/v1/txt2img"
    
    payload = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "steps": steps,
        "cfg_scale": cfg_scale,
        "width": 512,
        "height": 512,
        "sampler_name": "DPM++ 2M Karras"
    }
    
    try:
        response = requests.post(url, json=payload, timeout=300)
        if response.status_code == 200:
            result = response.json()
            return result['images'][0]  # Base64 encoded image
        else:
            print(f"API Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Example usage (uncomment to test)
# image_b64 = generate_image_api("a beautiful landscape with mountains and lake, high quality")
# if image_b64:
#     print("Image generated successfully!")

## 8. Utility Functions

In [ ]:
# Backup models to S3
def backup_to_s3(bucket_name=None):
    """Backup models and outputs to S3"""
    
    if not bucket_name:
        # Try to get bucket name from CloudFormation
        try:
            import boto3
            cf = boto3.client('cloudformation')
            response = cf.describe_stacks(StackName='stable-diffusion-webui')
            outputs = response['Stacks'][0]['Outputs']
            bucket_name = next(o['OutputValue'] for o in outputs if o['OutputKey'] == 'S3BucketName')
        except:
            print("Could not find S3 bucket. Please specify bucket_name parameter.")
            return
    
    print(f"Backing up to S3 bucket: {bucket_name}")
    
    # Backup models
    !aws s3 sync {webui_path}/models/ s3://{bucket_name}/models/ --exclude "*.tmp"
    
    # Backup outputs
    !aws s3 sync {webui_path}/outputs/ s3://{bucket_name}/outputs/
    
    print("Backup completed!")

# Uncomment to backup
# backup_to_s3()

In [ ]:
# Clean up temporary files
def cleanup_temp_files():
    """Clean up temporary files to free space"""
    
    print("Cleaning up temporary files...")
    
    # Clear pip cache
    !pip cache purge
    
    # Clear huggingface cache
    !rm -rf ~/.cache/huggingface/
    
    # Clear temporary model files
    !find {webui_path} -name "*.tmp" -delete
    
    print("Cleanup completed!")
    
    # Show disk usage
    !df -h /home/ec2-user/SageMaker

# Uncomment to cleanup
# cleanup_temp_files()

## 9. Quick Start Guide

### For Inpainting with ControlNet:

1. **Access WebUI**: Click the link above to open WebUI
2. **Select Model**: Choose `sd1_5-epiCRealism.safetensors`
3. **Go to img2img → Inpaint tab**
4. **Upload your image** and use brush to mask areas
5. **Enable ControlNet**:
   - Upload control image
   - Select `control_v11p_sd15_inpaint` model
   - Set control mode to "ControlNet is more important"
6. **Generate** your inpainted image!

### Tips:
- Use descriptive prompts for better results
- Adjust denoising strength (0.3-0.8 works well)
- Try different sampling methods
- Save your favorite settings as styles